In [45]:
import numpy as np
import glob

def load_data(path):
    io_files = glob.glob(path+"/io/*.txt")
    nio_files = glob.glob(path+"/nonio/*.txt")

    X = []
    y = []
    for iof in io_files:
            f = open(iof, 'r', encoding="utf8")
            line = f.read()
            #line = line[:len(line)/2]
            X.append(line)
            y.append(1)
            f.close()

    for niof in nio_files:
            f = open(niof, 'r', encoding="utf8")
            line = f.read()
            #line = line[:len(line)/2]
            X.append(line)
            y.append(0)
            f.close()

    y = np.array(y)
    
    return X, y

def generate_token_sequence(X_corpus, word_dict, token):
    import re

    token_pattern = re.compile(token)
    X = []
    i=0
    for sentence in X_corpus:
        split = token_pattern.findall(sentence)
        seq = []
        for word in split:
            try:
                seq.append(word_dict[word])
            except KeyError:
                continue
        X.append(seq)

    return np.asarray(X)

In [25]:
X, y = load_data("./dataset")

In [51]:
for i,x in enumerate(X):
    print('File',i)
    print(x)

File 0
main()
{
    void* buff = calloc(1024, 1);
    FILE* fh = fopen("filename.txt", "rt");
    fwrite(buff, 1024,1, fh);
    fclose(fh);
}
File 1
main() 
{
   FILE *fp;
   char buff[255];

   fp = fopen("/tmp/test.txt", "r");
   fscanf(fp, "%s", buff);
   printf("1 : %s\n", buff );

   fgets(buff, 255, (FILE*)fp);
   printf("2: %s\n", buff );
   
   fgets(buff, 255, (FILE*)fp);
   printf("3: %s\n", buff );
   fclose(fp);
}
File 2
int main( )               
{
    float m, n ;
    printf ( "\nEnter some number for finding square \n");
    scanf ( "%f", &m ) ;
    n = square ( m ) ;                      
    printf ( "\nSquare of the given number %f is %f",m,n );
}
File 3
int main(int argc, char* argv[] )
{
    printf( "argc = %d\n", argc );
    for( int i = 0; i < argc; ++i ) {
        printf( "argv[ %d ] = %s\n", i, argv[ i ] );
    }
}


In [6]:
y

array([1, 1, 0, 0])

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(lowercase=True, max_df=1.0, min_df=1, binary=False, token_pattern=token, ngram_range=(1,1))
cv.fit(X)
X_tr = cv.transform(X)
X_tr.shape

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_tr, y)
#print(clf.score(X_tr,y))

word = cv.get_feature_names()
coef = clf.coef_.flatten()

for i,word in enumerate(word):
    if coef[i] > 0:
        print("%s\t%f" %(word, coef[i]))
    else:
        continue

1	0.161364
1024	0.137276
2	0.024088
255	0.072265
3	0.024088
buff	0.305895
calloc	0.068638
fclose	0.092726
fgets	0.048177
fh	0.205914
file	0.140903
filename	0.068638
fopen	0.092726
fp	0.144530
fscanf	0.024088
fwrite	0.068638
main	0.009238
r	0.024088
rt	0.068638
s	0.057402
tmp/test	0.024088
txt	0.092726
void	0.068638


In [39]:
# LSTM
cv.fit(X)
word_dict = cv.vocabulary_
dict_len = len(cv.get_feature_names())
word_feature = cv.get_feature_names()
  
print('Words :', len(cv.get_feature_names()))
  
print('Generate token sequence...')
X_train = generate_token_sequence(X, word_dict, token)

from keras.preprocessing.sequence import pad_sequences

print('Generate pad sequences...')
print('Pad sequences (samples x time)')
maxlen=100
x_train = pad_sequences(X_train, maxlen=maxlen, padding='pre', truncating='pre', value=0)
print('x_train shape:', x_train.shape)

del X_train

print('Generate one hot...')

def generate_one_hot(X):
    X_one_hot = []
  
    for sent in X:
        sent_reverse = []
        for idx in sent:
            sent_reverse.append(word_feature[idx])
        sent_transform = tf_vectorizer.transform(sent_reverse).todense()
        X_one_hot.append(sent_transform)
        del sent_transform
    return np.array(X_one_hot)

X_train_hot = generate_one_hot(x_train)
del x_train

Words : 47
Generate token sequence...
Generate pad sequences...
Pad sequences (samples x time)
x_train shape: (4, 100)
Generate one hot...


In [40]:
X_train_hot.shape

(4, 100, 47)

In [44]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Flatten
from keras.layers import Embedding

hidden_neurons = 100

print('Build model...')
model = Sequential()
# model.add(Embedding(dict_len, 100))
model.add(LSTM(output_dim=hidden_neurons, input_dim=dict_len))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  
print(model.summary())

print('Train...')
hist = model.fit(X_train_hot, y, epochs=100, verbose=0, shuffle=False)

y_pred = model.predict(X_train_hot)

Build model...


C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  if sys.path[0] == '':
C:\Users\Anneke Hidayat\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(None, 47), units=100)`
  if sys.path[0] == '':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               59200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 59,301
Trainable params: 59,301
Non-trainable params: 0
_________________________________________________________________
None
Train...


In [22]:
y_pred

array([[0.99857235],
       [0.9985917 ],
       [0.00106561],
       [0.00106516]], dtype=float32)

In [23]:
y

array([1, 1, 0, 0])